# Homework 5 - Explore California and Nevada with graphs
---
Here we import every library we'll need.

In [1]:
import networkx as nx
import numpy as np

---
# Graph class
Here we're going to create our graph class which we're going to use for basically every task. Since we'll use a lot of nodes and every node shouldn't be connected to many edges (considering that our graph represents a map of roads) we'll use adjacency list to represent the graph. An adjacency matrix would take too much memory and most of the entries would be zero, so it would be really inefficient.

In [2]:
# This class is going to represent a graph, so that we can organize all the implementation and methods of
# our data structure in an organized way
class Graph:
    
    # This constructur takes as parameter the number of nodes
    def __init__(self, size):
        self._nodes = [[] for _ in range(size)]
        
    # This method adds an edge to the graph, together with an optional dict of weights
    # The edge is given as a tuple of nodes
    def add_edge(self, edge, weights = None):
        to_add = edge[1]
        if(not weights is None):
            to_add = (edge[1], weights)
        self._nodes[edge[0]].append(to_add)
        
    # This method gets all neighbours of a node and given weights of the edges connecting them to the node
    def get_neighbours(self, node, features = None):
        if(features is None):
            return([single_node[0] for single_node in self._nodes[node]])
        to_return = []
        for neighbour in self._nodes[node]:
            to_return.append((neighbour[0], dict((k, neighbour[1][k]) for k in features if k in neighbour[1])))
        return(to_return)
    
    # This method get an edge from the graph together with the specified weights
    # If the edge doesn't exist it returns None
    def get_edge(self, node_one, node_two, features = None):
        for neighbour in self._nodes[node_one]:
            if(neighbour[0] == node_two):
                if(features is None):
                    return((node_one, node_two))
                return((node_one, node_two, dict((k, neighbour[1][k]) for k in features if k in neighbour[1])))
        return(None)